In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.utils.prune as prune

from scipy import io
import os

/Users/goldenyoo/miniforge3/envs/mac_cpu/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
mat_file = io.loadmat('/Users/goldenyoo/Library/Mobile Documents/com~apple~CloudDocs/BioCAS_prepare/Python_code/Data_center/one_dx/Calib_data_1.mat')


In [ ]:
K1 = mat_file['K1']
A1 = mat_file['A1']

K2 = mat_file['K2']
A2 = mat_file['A2']

# K1 = np.swapaxes(K1,0,2)
# K1 = np.swapaxes(K1,1,2)
# A1 = np.swapaxes(A1,0,2)
# A1 = np.swapaxes(A1,1,2)

# K2 = np.swapaxes(K2,0,2)
# K2 = np.swapaxes(K2,1,2)
# A2 = np.swapaxes(A2,0,2)
# A2 = np.swapaxes(A2,1,2)


Y1 = mat_file['Y1']
Y2 = mat_file['Y2']

In [ ]:
k1 = torch.FloatTensor(K1)

In [2]:
n_class = 2
n_hidden = 5
dtype = torch.float

class TextLSTM(nn.Module):
  def __init__(self):
    super(TextLSTM, self).__init__()

    self.lstm = nn.LSTM(input_size=n_class, hidden_size=n_hidden, dropout=0.3)
    self.W = nn.Parameter(torch.randn([n_hidden, n_class]).type(dtype))
    self.b = nn.Parameter(torch.randn([n_class]).type(dtype))
    # self.Softmax = nn.Softmax(dim=1) # softmax 포함되어있다고 하지 않았나? CrossEntropyLoss에...

  def forward(self, hidden_and_cell, X):
    X = X.transpose(0, 1)
    # outputs, hidden = self.lstm(X, hidden_and_cell)
    # outputs = outputs[-1]  # 최종 예측 Hidden Layer
    print(X.size())
    outputs, (h_n,c_n) = self.lstm(X, hidden_and_cell)
    # print(h_n.size())
    outputs = h_n[-1]  # 최종 예측 Hidden Layer

    model = torch.mm(outputs, self.W) + self.b  # 최종 예측 최종 출력 층
    return model

In [3]:
model = TextLSTM()

module_1 = model.lstm

/Users/goldenyoo/miniforge3/envs/mac_cpu/lib/python3.8/site-packages/torch/nn/modules/rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [6]:
list(module_1.named_parameters())

[('weight_ih_l0',
  Parameter containing:
  tensor([[ 0.2912, -0.1798],
          [ 0.4055, -0.4347],
          [ 0.2762,  0.0449],
          [-0.4446, -0.1436],
          [-0.3310,  0.3607],
          [ 0.2106,  0.1642],
          [-0.2545, -0.2307],
          [ 0.2995,  0.2674],
          [-0.1041, -0.1720],
          [-0.3556,  0.1214],
          [-0.1920, -0.4170],
          [ 0.1555,  0.3341],
          [ 0.1053,  0.3932],
          [ 0.0947, -0.3684],
          [ 0.0451,  0.4370],
          [ 0.3150, -0.1369],
          [ 0.1610,  0.0980],
          [ 0.4422,  0.0916],
          [ 0.0455,  0.1501],
          [-0.3389, -0.2318]], requires_grad=True)),
 ('weight_hh_l0',
  Parameter containing:
  tensor([[-0.1896,  0.1681, -0.2762,  0.4190,  0.1899],
          [-0.0782, -0.1045, -0.1679,  0.1726, -0.3120],
          [-0.3348, -0.2711, -0.1111, -0.3023, -0.2008],
          [-0.3514,  0.2660,  0.2612, -0.3102, -0.2143],
          [ 0.3105,  0.2031, -0.0816, -0.3132, -0.1966],
        

In [8]:
prune.random_unstructured(module_1, name="weight_ih_l0", amount=0.3)
list(module_1.named_parameters())

[('weight_hh_l0',
  Parameter containing:
  tensor([[-0.1896,  0.1681, -0.2762,  0.4190,  0.1899],
          [-0.0782, -0.1045, -0.1679,  0.1726, -0.3120],
          [-0.3348, -0.2711, -0.1111, -0.3023, -0.2008],
          [-0.3514,  0.2660,  0.2612, -0.3102, -0.2143],
          [ 0.3105,  0.2031, -0.0816, -0.3132, -0.1966],
          [ 0.4293,  0.1276,  0.2366, -0.1929, -0.3545],
          [ 0.3165,  0.2339,  0.1177, -0.4192, -0.2537],
          [-0.1396,  0.0333,  0.1935,  0.1796,  0.1987],
          [ 0.1791,  0.1760, -0.1893,  0.4311, -0.1467],
          [ 0.2123, -0.4436, -0.3329, -0.3002,  0.2147],
          [ 0.2450,  0.0666,  0.0849,  0.2691, -0.0472],
          [-0.0140,  0.1025,  0.1138, -0.4273,  0.3973],
          [-0.2059,  0.0974, -0.4086, -0.2739,  0.1365],
          [-0.0364,  0.1302,  0.0849, -0.3353, -0.0420],
          [-0.0608,  0.4346,  0.1932,  0.0083,  0.4157],
          [-0.2471,  0.1273,  0.2255,  0.0109, -0.3231],
          [ 0.2506, -0.1825,  0.3534, -0.3917,

In [9]:
list(module_1.named_buffers())

[('weight_ih_l0_mask',
  tensor([[0., 1.],
          [1., 1.],
          [0., 1.],
          [1., 1.],
          [0., 1.],
          [0., 1.],
          [1., 1.],
          [1., 0.],
          [1., 0.],
          [1., 1.],
          [0., 1.],
          [1., 0.],
          [1., 1.],
          [0., 1.],
          [1., 1.],
          [0., 1.],
          [1., 1.],
          [1., 1.],
          [1., 0.],
          [0., 1.]]))]

In [12]:
module_1.weight_ih_l0

tensor([[ 0.0000, -0.1798],
        [ 0.4055, -0.4347],
        [ 0.0000,  0.0449],
        [-0.4446, -0.1436],
        [-0.0000,  0.3607],
        [ 0.0000,  0.1642],
        [-0.2545, -0.2307],
        [ 0.2995,  0.0000],
        [-0.1041, -0.0000],
        [-0.3556,  0.1214],
        [-0.0000, -0.4170],
        [ 0.1555,  0.0000],
        [ 0.1053,  0.3932],
        [ 0.0000, -0.3684],
        [ 0.0451,  0.4370],
        [ 0.0000, -0.1369],
        [ 0.1610,  0.0980],
        [ 0.4422,  0.0916],
        [ 0.0455,  0.0000],
        [-0.0000, -0.2318]], grad_fn=<MulBackward0>)